In [7]:
import os, json
from collections import OrderedDict

import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
with open('Daten/Unternehmen_prepocessed.json', 'r', encoding='utf-8') as files:
    data = json.load(files)

## Vectorize Text

In [9]:
docs = [corp['explicit_lemmatization'] for corp in data]

## Get the words with the highest TfIdf Value

Function to split lemmatized description into earlier defined tokens

In [10]:
def splitter(txt):
    return txt.split(' ')

### Create TfIdf Vector

In [11]:
tf_vectorizer = TfidfVectorizer(
    tokenizer=splitter
    )
tf_vector = tf_vectorizer.fit_transform(docs)

In [12]:
max_values = tf_vector.argmax(axis=0)
max_values.shape

(1, 41168)

## Create TfIdf list for all sectors

In [13]:
tf_vector.shape[1]

41168

Create a dictionary for the tf_idf values for each word. Each entry will be indexed by the word. That way it can easily be searched through to built a thesaurus. 

In [14]:
idf_dict = {}

for word_id in range(0, tf_vector.shape[1]):
    idf_dict[tf_vectorizer.get_feature_names_out()[word_id]] = {"value" : tf_vector[max_values[0, word_id], word_id],
                     "sector": data[max_values[0, word_id]]["sector"], "nr": max_values[0, word_id]}
    if word_id % 500 == 0:
        print(f'{word_id} words done.')

0 words done.
500 words done.
1000 words done.
1500 words done.
2000 words done.
2500 words done.
3000 words done.
3500 words done.
4000 words done.
4500 words done.
5000 words done.
5500 words done.
6000 words done.
6500 words done.
7000 words done.
7500 words done.
8000 words done.
8500 words done.
9000 words done.
9500 words done.
10000 words done.
10500 words done.
11000 words done.
11500 words done.
12000 words done.
12500 words done.
13000 words done.
13500 words done.
14000 words done.
14500 words done.
15000 words done.
15500 words done.
16000 words done.
16500 words done.
17000 words done.
17500 words done.
18000 words done.
18500 words done.
19000 words done.
19500 words done.
20000 words done.
20500 words done.
21000 words done.
21500 words done.
22000 words done.
22500 words done.
23000 words done.
23500 words done.
24000 words done.
24500 words done.
25000 words done.
25500 words done.
26000 words done.
26500 words done.
27000 words done.
27500 words done.
28000 words done

In [15]:
#idf_dict

In [16]:
my_list = OrderedDict(sorted(idf_dict.items(), key=lambda x: x[1]['value'], reverse=True))

In [17]:
my_list

OrderedDict([('sapiens',
              {'value': 0.9620951197379793,
               'sector': 'Technology',
               'nr': 4439}),
             ('rtl',
              {'value': 0.9593074983770934,
               'sector': 'Communication Services',
               'nr': 9652}),
             ('rakuten',
              {'value': 0.9577919815099941,
               'sector': 'Consumer Cyclical',
               'nr': 8608}),
             ('netapp',
              {'value': 0.9542382163958185,
               'sector': 'Technology',
               'nr': 3628}),
             ('orbis',
              {'value': 0.9532792573146146,
               'sector': 'Technology',
               'nr': 9760}),
             ('blackbaud',
              {'value': 0.9468649799558335,
               'sector': 'Technology',
               'nr': 1737}),
             ('ccc',
              {'value': 0.9434834018389193,
               'sector': 'Technology',
               'nr': 5501}),
             ('itmedia',
      

In [18]:
for entry in my_list.items():
    entry[1]['nr'] = int(entry[1]['nr'])

## Safe list as Json File

In [19]:
with open('Daten/Tf_idf_dict.json', 'w') as file:
    json.dump(my_list, file)

## Create Thesaurus Search function

In [20]:
with open('Daten/Tf_idf_dict.json', 'r') as file:
    data = json.load(file, object_pairs_hook=OrderedDict)

In [21]:
len(data)

41168

In [22]:
data

OrderedDict([('sapiens',
              OrderedDict([('value', 0.9620951197379793),
                           ('sector', 'Technology'),
                           ('nr', 4439)])),
             ('rtl',
              OrderedDict([('value', 0.9593074983770934),
                           ('sector', 'Communication Services'),
                           ('nr', 9652)])),
             ('rakuten',
              OrderedDict([('value', 0.9577919815099941),
                           ('sector', 'Consumer Cyclical'),
                           ('nr', 8608)])),
             ('netapp',
              OrderedDict([('value', 0.9542382163958185),
                           ('sector', 'Technology'),
                           ('nr', 3628)])),
             ('orbis',
              OrderedDict([('value', 0.9532792573146146),
                           ('sector', 'Technology'),
                           ('nr', 9760)])),
             ('blackbaud',
              OrderedDict([('value', 0.9468649799558335),
   

In [23]:
def search_thesaurus(search_word, data=data):
    # Check if word exists in 'database'
    if search_word in data:
        print(f"{search_word} has a TfIdf Value of {data[search_word]['value']} and belongs to the {data[search_word]['sector']} sector.")
    else:
        print(f"{search_word} can not be found in the database.")

In [24]:
search_thesaurus(input("Please enter your search term:\n"))

Please enter your search term:
test
test has a TfIdf Value of 0.7398857417428469 and belongs to the Technology sector.


## Search for the most important words in each category

In [26]:
double_sort = OrderedDict(sorted(idf_dict.items(), key=lambda x: (x[1]['sector'], -x[1]['value'])))

In [27]:
double_sort

OrderedDict([('film',
              {'value': 0.9191986358765676,
               'sector': 'Basic Materials',
               'nr': 8513}),
             ('itaconix',
              {'value': 0.8432190768613468,
               'sector': 'Basic Materials',
               'nr': 1060}),
             ('paper',
              {'value': 0.8207508384067129,
               'sector': 'Basic Materials',
               'nr': 8435}),
             ('pile',
              {'value': 0.812007374030165,
               'sector': 'Basic Materials',
               'nr': 8689}),
             ('lithium',
              {'value': 0.8001928220329629,
               'sector': 'Basic Materials',
               'nr': 6443}),
             ('titanium',
              {'value': 0.7874809093796723,
               'sector': 'Basic Materials',
               'nr': 8733}),
             ('uranium',
              {'value': 0.7743989332820765,
               'sector': 'Basic Materials',
               'nr': 9318}),
             

In [53]:
prev_sector = "Nothing"
counter = 0

pd_df = pd.DataFrame()
data = {'Word': [], 'Value': []}

sector_list = []

for i, word in enumerate(double_sort.items()):
    sector = word[1]['sector']
    if sector != prev_sector and prev_sector != "Nothing":
        counter = 0
        df = pd.DataFrame(data)
        pd_df = pd.concat([pd_df, df], axis = 1)
        data = {'Word': [], 'Value': []}
    if i == len(double_sort.items())-1:
        df = pd.DataFrame(data)
        pd_df = pd.concat([pd_df, df], axis = 1)
    if counter < 10:
        print(counter)
        data['Word'].append(word[0])
        data['Value'].append(word[1]['value'])
        #print(word[0], word[1])
        counter += 1
    prev_sector = sector
    sector_list.append(sector)

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9


In [54]:
sector_list = list(dict.fromkeys(sector_list))

In [58]:
pd_df.columns = pd.MultiIndex.from_product([sector_list, ['Word', 'Value']])

# Show highest TfIdf Values for each sector

In [62]:
pd.set_option('display.max_columns', None)
pd_df

Basic Materials           Communication Services            \
             Word     Value                   Word     Value   
0            film  0.919199                    rtl  0.959307   
1        itaconix  0.843219                itmedia  0.931728   
2           paper  0.820751                 lifull  0.915377   
3            pile  0.812007                   ooma  0.907455   
4         lithium  0.800193                   imax  0.866856   
5        titanium  0.787481                    iij  0.845762   
6         uranium  0.774399                marchex  0.840480   
7          wacker  0.770644               doordash  0.838935   
8        brenntag  0.705959               litalico  0.833379   
9            pole  0.703941                 google  0.807665   

  Consumer Cyclical           Consumer Defensive                 Energy  \
               Word     Value               Word     Value         Word   
0           rakuten  0.957792              sugar  0.847620        egdon   
1           mercari  0.928450               weis  0.838955         coal   
2             meliá  0.896822              chegg  0.819145     drilling   
3         nordstrom  0.883046             coffee  0.806315        vista   
4            hilton  0.878707                tea  0.780439     energean   
5              pool  0.874220           lubrajel  0.758586    silverbow   
6            curves  0.872383            monster  0.753741       silica   
7             denny  0.870067            tobacco  0.750917       laredo   
8            4sight  0.865943             lawson  0.724174  compression   
9          skechers  0.847803               reed  0.716093          pdc   

            Financial Services           Healthcare           Industrials  \
      Value               Word     Value       Word     Value        Word   
0  0.779634                ofs  0.918089    nextgen  0.927108        trex   
1  0.719983              qatar  0.862350      veeva  0.919316         imi   
2  0.714626               naga  0.861745        cue  0.896290        arlo   
3  0.708043            invesco  0.849809   graphene  0.883411  consulting   
4  0.694511                svf  0.839870      nexus  0.882708   container   
5  0.668714           maebashi  0.828109        htg  0.792459        tape   
6  0.664157               fund  0.820280        igm  0.783704       byrna   
7  0.660742           tetragon  0.816771     trxade  0.783503     diamond   
8  0.646183               pawn  0.815639    impella  0.777785         jac   
9  0.643953            burtech  0.806700        aro  0.777510     ceiling   

              Real Estate           Technology                Utilities  \
      Value          Word     Value       Word     Value           Word   
0  0.920542    aroundtown  0.832764    sapiens  0.962095       hydrogen   
1  0.912601         onsen  0.825289     netapp  0.954238  contourglobal   
2  0.891280        ichigo  0.814646      orbis  0.953279     excelerate   
3  0.881872       katitas  0.801124  blackbaud  0.946865         ameren   
4  0.868026       helical  0.791609        ccc  0.943483       clearway   
5  0.823940  purplebricks  0.784104       btob  0.903760          saibu   
6  0.822741          stag  0.770515     hennge  0.900128       dominion   
7  0.813099         aimco  0.754892    zendesk  0.900030            pne   
8  0.805602      patrizia  0.752548   fabasoft  0.891860      renewable   
9  0.797292           bmo  0.741360       zoom  0.891435            gas   

             
      Value  
0  0.712654  
1  0.702817  
2  0.692075  
3  0.669005  
4  0.662890  
5  0.656893  
6  0.632377  
7  0.626896  
8  0.625533  
9  0.623550